In [1]:
# unicode 오류 원인 = 파일 이름이 한국어로 되어 있어서 !!!!!!

import tensorflow as tf
#이미지 데이터셋 생성 (숫자화)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    './dataset/',
     # 이미지 사이즈를 64크기로 맞춰준다
     image_size=(64,64),
     # 이미지 2만개를 다 넣지않고 batch숫자만큼 넣는다
     batch_size=64,
     # 검사
     subset = 'training',
     # 데이터를 20% 쪼갠다
     validation_split=0.2,
     seed=1234
)
# training = 80%, validation = 20% 
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    './dataset/',
     # 이미지 사이즈를 64크기로 맞춰준다
     image_size=(64,64),
     # 이미지 2만개를 다 넣지않고 batch숫자만큼 넣는다
     batch_size=64,
     # 검사
     subset = 'validation',
     # 데이터를 20% 쪼갠다
     validation_split=0.2,
     seed=1234
)
print(train_ds)

# 0~1값으로 압축
def preprocessing(i, answer):
    i = tf.cast(i/255.0, tf.float32)
    return i, answer

# map은 모든 데이터에 함수를 적용
train_ds = train_ds.map(preprocessing)
val_ds = val_ds.map(preprocessing)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<_BatchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [2]:
# 이전 model 재활용
model = tf.keras.Sequential([
    # (사진 뒤집기)
    # epochs동안 여러 다른 각도 사진 학습
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal', input_shape = (64,64,3)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),

    tf.keras.layers.Conv2D(32,(3,3), padding="same", activation="relu"), # 컬러 사진이기 때문에 1 -> 3 으로 수정
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3), padding="same", activation="relu"), # 컬러 사진이기 때문에 1 -> 3 으로 수정
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(128,(3,3), padding="same", activation="relu"), # 컬러 사진이기 때문에 1 -> 3 으로 수정
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])

# 모델 아웃라인 출력하기
model.summary()
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
# validation_data= epoch1회 끝날때 마다 채점해준다.
model.fit(train_ds, validation_data = val_ds,epochs=5)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip (RandomFlip)    (None, 64, 64, 3)         0         
                                                                 
 random_rotation (RandomRot  (None, 64, 64, 3)         0         
 ation)                                                          
                                                                 
 random_zoom (RandomZoom)    (None, 64, 64, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 32, 32, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        1

In [ ]:
# 실험 자동화